In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import MELODIC
%matplotlib inline

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects_sim.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [10]:
def apply_melodic(in_tuple):
    sub, exp, run = in_tuple
    
    sub_path = home_dir + 'analysis/' + exp + '_8mm/' + sub + '/preproc/run_' + run + '/'
    
    if os.path.exists(sub_path):
        scan = sub_path + 'unsmoothed_timeseries.nii.gz'
        mask = sub_path + 'functional_mask.nii.gz'
        out_dir = sub_path + 'melodic'
        
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)
        
        if os.listdir(out_dir) == []: #directory empty
            melodic_setup = MELODIC(in_files = scan, no_bet = True, bg_threshold = 3, tr_sec = 1.5,
                                   mm_thresh = .5, out_stats = True, report = True, mask = mask,
                                   out_dir = out_dir)
            melodic_setup.run()

In [ ]:
in_tuples = []
for sub in sub_list:
    for exp in ['ser','sim']:
        for run in range(1,4):
            in_tuples.append((sub,exp,str(run)))

pool = multiprocessing.Pool(processes = 24)
pool.map(apply_melodic,in_tuples)